<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Convolutional%20Neural%20Networks/cnn_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
w_dir = '/content/drive/My Drive/Colab/D2L.AI/'
sys.path.append(w_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing Libraries

In [2]:
!pip uninstall mxnet && pip install mxnet-cu100

Importing Libraries

In [0]:
import mxnet as mx
import d2l
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import loss as gloss, nn
import time

LeNet

In [0]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, padding=2, activation='sigmoid'),
        nn.AvgPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
        nn.AvgPool2D(pool_size=2, strides=2),
        # Dense will transform the input of the shape (batch size, channel, height, width) 
        # into the input of the shape (batch size, channel * height * width) automatically by default
        nn.Dense(120, activation='sigmoid'),
        nn.Dense(84, activation='sigmoid'),
        nn.Dense(10))
  

In [5]:
X = nd.random.uniform(shape=(1, 1, 28, 28))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 6, 28, 28)
pool0 output shape:	 (1, 6, 14, 14)
conv1 output shape:	 (1, 16, 10, 10)
pool1 output shape:	 (1, 16, 5, 5)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


Data Acquisition and Training

In [0]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [7]:
def try_gpu():
  try:
    ctx = mx.gpu()
    _ = nd.array([1], ctx=ctx)
  except mx.base.MXNetError:
    ctx = mx.cpu()
  return ctx

ctx = try_gpu()
ctx

gpu(0)

In [0]:
def evaluate_accuracy(data_iter, net, ctx):
  acc_sum, n = nd.array([0], ctx=ctx), 0
  for X, y in data_iter:
    X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
    acc_sum += (net(X).argmax(axis=1) == y).sum()
    n += y.size
  return acc_sum.asscalar() / n

In [0]:
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
  print('training on', ctx)
  loss = gloss.SoftmaxCELoss()
  for epoch in range(num_epochs):
    train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
    for X, y in train_iter:
      X, y = X.as_in_context(ctx), y.as_in_context(ctx)
      with autograd.record():
        y_hat = net(X)
        l = loss(y_hat, y).sum()
      l.backward()
      trainer.step(batch_size)
      y = y.astype('float32')
      train_l_sum += l.asscalar()
      train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
      n += y.size
    test_acc = evaluate_accuracy(test_iter, net, ctx)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec' % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc, time.time() - start))

In [10]:
lr, num_epochs = 0.9, 10
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.3196, train acc 0.098, test acc 0.100, time 4.9 sec
epoch 2, loss 2.2561, train acc 0.134, test acc 0.430, time 4.7 sec
epoch 3, loss 1.1088, train acc 0.552, test acc 0.672, time 4.7 sec
epoch 4, loss 0.8032, train acc 0.684, test acc 0.721, time 4.6 sec
epoch 5, loss 0.6836, train acc 0.732, test acc 0.756, time 4.7 sec
epoch 6, loss 0.6136, train acc 0.760, test acc 0.781, time 4.7 sec
epoch 7, loss 0.5649, train acc 0.781, test acc 0.794, time 4.7 sec
epoch 8, loss 0.5330, train acc 0.793, test acc 0.818, time 4.7 sec
epoch 9, loss 0.4996, train acc 0.808, test acc 0.827, time 4.9 sec
epoch 10, loss 0.4748, train acc 0.820, test acc 0.836, time 4.5 sec


In [11]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True, ctx=mx.cpu(), init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, mx.cpu(), num_epochs)

training on cpu(0)
epoch 1, loss 2.3207, train acc 0.101, test acc 0.100, time 26.3 sec
epoch 2, loss 2.1631, train acc 0.173, test acc 0.489, time 24.6 sec
epoch 3, loss 1.0340, train acc 0.585, test acc 0.676, time 26.0 sec
epoch 4, loss 0.8111, train acc 0.685, test acc 0.735, time 26.8 sec
epoch 5, loss 0.6881, train acc 0.732, test acc 0.758, time 24.6 sec
